In [ ]:
import pandas_datareader.data as web
import talib
import html5lib
import yfinance as yf #yahoo
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
import numpy as np
import pandas
import ta
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#create indicators and add them to the DF
def indicators(df,close,time=[3,5,10,20,60,90,120,252],returns=True):
    #lowercase all column names
    close = close.lower()
    df.columns = df.columns.str.lower() 
    
    #calculating returns
    if returns==True:
        #daily returns
        df["ret_d"] = df.loc[:,close].pct_change()
        #log returns
        #cummulative returns
        df["ret_cum"] = (1+df.loc[:,close].pct_change()).cumprod()-1
    
    #calculating indicators
    for t in time:
        df[f'sma_{t}'] = talib.SMA(df.loc[:,close], timeperiod=t)
        df[f'wma_{t}'] = talib.WMA(df.loc[:,close], timeperiod=t)
        df[f'trima_{t}'] = talib.TRIMA(df.loc[:,close], timeperiod=t)
        df[f'ema_{t}'] = talib.EMA(df.loc[:,close], timeperiod=t)
        df[f'dema_{t}'] = talib.DEMA(df.loc[:,close], timeperiod=t)
        df[f'tema_{t}'] = talib.TEMA(df.loc[:,close], timeperiod=t)
        df[f'kama_{t}'] = talib.KAMA(df.loc[:,close], timeperiod=t)

    df.loc[:,'sar'] = talib.SAR(df.high,df.low,
                                acceleration=.02,
                                maximum=.2)
    
    df.loc[:,'adx'] = talib.ADX(df.high,df.low,df.loc[:,close],
                                timeperiod=14)
    
    df['adxr'] = talib.ADXR(df.high,
                        df.low,
                        df.loc[:,close],
                        timeperiod=14)
    
    df['rsi'] = talib.RSI(df.loc[:,close],
                          timeperiod=14)
                                     
    slowk, slowd = talib.STOCH(df.high,df.low,df.loc[:,close],
                           fastk_period=14,
                           slowk_period=3,
                           slowk_matype=0,
                           slowd_period=3,
                           slowd_matype=0)
    df['stoch'] = slowd / slowk
    
    df['aroon_osc'] = talib.AROONOSC(high=df.high,low=df.low,
                                     timeperiod=14)
    
    df['cci'] = talib.CCI(high=df.high,
                          low=df.low,
                          close=df.loc[:,close],
                          timeperiod=14)

    df['bop'] = talib.BOP(open=df.open,
                      high=df.high,
                      low=df.low,
                      close=df.loc[:,close])
    

    df['cmo'] = talib.CMO(df.loc[:,close],
                          timeperiod=14)

    df['mfi'] = talib.MFI(df.high, 
                      df.low, 
                      df.loc[:,close], 
                      df.volume, 
                      timeperiod=14)
  

    df['adosc'] = talib.ADOSC(df.high,
                          df.low,
                          df.loc[:,close],
                          df.volume,
                          fastperiod=3,
                          slowperiod=10)

    df['obv'] = talib.OBV(df.loc[:,close],
                      df.volume)
    
    df['atr'] = talib.ATR(df.high,
                      df.low,
                      df.loc[:,close],
                      timeperiod=14)

In [ ]:
#download data from Yahoo Finance
start_date = "1980-01-01"
end_date = "2021-12-31"
sp500 = yf.download("^gspc",start=start_date)
nasdaq = yf.download("^ixic",start=start_date)
vix = yf.download("^VIX",start=start_date) #S&P500 volatility
sp500_futures = yf.download("ES=F",start=start_date)
nasdaq_futures = yf.download("NQ=F",start=start_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
#display(sp500_futures)
#display(nasdaq_futures)
#display(vix)
sp500.name = "S&P 500"
nasdaq.name = "NASDAQ"
sp500_futures.name = "S&P 500 Futures"
nasdaq_futures.name = "NASDAQ Futures"
vix.name = "VIX"

## Macroeconomic Indicators (FRED)

In [ ]:
#import macro data from FRED
int_rate_2_10 = pd.read_csv("./data/T10Y2Y.csv",na_values='.', header=0, parse_dates=['date'], index_col='date', names=['date','int_rate_2_10'])
int_rate_2 = pd.read_csv("./data/DGS2.csv",na_values='.', header=0, parse_dates=['date'], index_col='date', names=['date','int_rate_2'])
int_rate_10 = pd.read_csv("./data/DGS10.csv",na_values='.', header=0, parse_dates=['date'], index_col='date', names=['date','int_rate_10'])
m1 = pd.read_csv("./data/WM1NS.csv", header=0, parse_dates=['date'], index_col='date', names=['date','m1'])
m2 = pd.read_csv("./data/M2SL.csv", header=0, parse_dates=['date'], index_col='date', names=['date','m2'])
m3 = pd.read_csv("./data/MABMM301USM189S.csv", header=0, parse_dates=['date'], index_col='date', names=['date','m3'])
curr_circulation = pd.read_csv("./data/CURRCIR.csv", header=0, parse_dates=['date'], index_col='date', names=['date','curr_circulation'])
savings_total = pd.read_csv("./data/PSAVERT.csv", header=0, parse_dates=['date'], index_col='date', names=['date','savings_total'])
gdp = pd.read_csv("./data/GDP.csv", header=0, parse_dates=['date'], index_col='date', names=['date','gdp'])
debt_gdp = pd.read_csv("./data/GFDEGDQ188S.csv", header=0, parse_dates=['date'], index_col='date', names=['date','debt_gdp'])
cpi = pd.read_csv("./data/MEDCPIM158SFRBCLE.csv", header=0, parse_dates=['date'], index_col='date', names=['date','cpi'])
payrolls_nonfarm = pd.read_csv("./data/PAYEMS.csv", header=0, parse_dates=['date'], index_col='date', names=['date','payrolls_nonfarm'])
unemp_rate = pd.read_csv("./data/UNRATE.csv", header=0, parse_dates=['date'], index_col='date', names=['date','unemp_rate'])
indust_prod = pd.read_csv("./data/INDPRO.csv", header=0, parse_dates=['date'], index_col='date', names=['date','indust_prod'])
hous_starts = pd.read_csv("./data/HOUST.csv", header=0, parse_dates=['date'], index_col='date', names=['date','hous_starts'])
hous_permits = pd.read_csv("./data/PERMIT.csv", header=0, parse_dates=['date'], index_col='date', names=['date','hous_permits'], engine='python')

In [ ]:
int_rate_2.isnull().sum()/int_rate_2.shape[0], int_rate_10.isnull().sum()/int_rate_10.shape[0]

(int_rate_2    0.042004
 dtype: float64,
 int_rate_10    0.042641
 dtype: float64)

In [ ]:
# clean interest rate
#int_rate_2_10.int_rate_2_10.replace(to_replace='.',value=np.nan,inplace=True)
# change to correct datatype
#int_rate_2_10.int_rate_2_10 = pd.to_numeric(int_rate_2_10.int_rate_2_10, errors='coerce')
print(int_rate_2_10.int_rate_2_10.value_counts())
print(f'\nNulls: {int_rate_2_10.int_rate_2_10.isnull().sum()/int_rate_2_10.shape[0]*100:.2f}%')

 0.17    81
 0.16    79
 1.04    77
 0.18    76
 1.05    72
         ..
-1.22     1
-1.96     1
-1.95     1
-2.04     1
 2.91     1
Name: int_rate_2_10, Length: 474, dtype: int64

Nulls: 4.20%


In [ ]:
# create a list of all indicators
macros_lst = [int_rate_2_10,m1,m2,m3,curr_circulation,savings_total,
              gdp,debt_gdp,cpi,payrolls_nonfarm,unemp_rate,indust_prod,
              hous_starts,hous_permits]

In [ ]:
# print the nulls, starting date, time frequency of each indicator
for df in macros_lst:
    print(f'''
{df.isnull().any()}
{df.index[0]}
{df.index[1] - df.index[0]}
''')


int_rate_2_10    True
dtype: bool
1976-06-01 00:00:00
1 days 00:00:00


m1    False
dtype: bool
1975-01-06 00:00:00
7 days 00:00:00


m2    False
dtype: bool
1959-01-01 00:00:00
31 days 00:00:00


m3    False
dtype: bool
1960-01-01 00:00:00
31 days 00:00:00


curr_circulation    False
dtype: bool
1917-08-01 00:00:00
31 days 00:00:00


savings_total    False
dtype: bool
1959-01-01 00:00:00
31 days 00:00:00


gdp    False
dtype: bool
1947-01-01 00:00:00
90 days 00:00:00


debt_gdp    False
dtype: bool
1966-01-01 00:00:00
90 days 00:00:00


cpi    False
dtype: bool
1983-01-01 00:00:00
31 days 00:00:00


payrolls_nonfarm    False
dtype: bool
1939-01-01 00:00:00
31 days 00:00:00


unemp_rate    False
dtype: bool
1948-01-01 00:00:00
31 days 00:00:00


indust_prod    False
dtype: bool
1919-01-01 00:00:00
31 days 00:00:00


hous_starts    False
dtype: bool
1959-01-01 00:00:00
31 days 00:00:00


hous_permits    False
dtype: bool
1960-01-01 00:00:00
31 days 00:00:00



In [ ]:
# create a dataframe with an index like S&P500 Futures (target)
daily_df = []
# fill NaN using the previous valid value
[daily_df.append(df.reindex(sp500_futures.index,method='ffill')) for df in macros_lst]
macro_df = pd.concat(daily_df,axis=1)
# fill remaining NaN
[col.fillna(method='bfill',inplace=True) for col_name,col in macro_df.iteritems()]
macro_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5472 entries, 2000-09-18 to 2022-05-12
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   int_rate_2_10     5472 non-null   float64
 1   m1                5472 non-null   float64
 2   m2                5472 non-null   float64
 3   m3                5472 non-null   float64
 4   curr_circulation  5472 non-null   float64
 5   savings_total     5472 non-null   float64
 6   gdp               5472 non-null   float64
 7   debt_gdp          5472 non-null   float64
 8   cpi               5472 non-null   float64
 9   payrolls_nonfarm  5472 non-null   int64  
 10  unemp_rate        5472 non-null   float64
 11  indust_prod       5472 non-null   float64
 12  hous_starts       5472 non-null   float64
 13  hous_permits      5472 non-null   float64
dtypes: float64(13), int64(1)
memory usage: 641.2 KB


# Merging DFs

In [ ]:
# create returns and indicators
[indicators(dfs,close='Adj Close', returns=False) for dfs in [nasdaq_futures,sp500_futures,vix]]
# add preffix to DF column names
df_list = [sp500_futures,nasdaq_futures,sp500,nasdaq,vix]
prefix = ['spf','nasf','sp','nas','vix']
for pref, df in zip(prefix, df_list):
    df.columns = [pref+'_'+col for col in df.columns]
#fill NaNs with the nearest value
[df.interpolate(method='bfill',inplace=True) for df in [sp500_futures,nasdaq_futures,sp500,nasdaq,vix]]
# check for NaN in DFs
[pd.isna(df).any().any() for df in df_list]

[None, None, None, None, None]

In [ ]:
#merge dfs and save to csv
final = pd.concat([sp500_futures,nasdaq_futures,sp500,nasdaq,vix,macro_df],axis=1)
final.drop(labels=['2016-10-10','2016-11-11'],inplace=True)
#final.fillna(method='pad',inplace=True)
final = final.loc[:,'2000-09-18']
# check that the indexes match
final.index.equals(sp500_futures.index)

True

In [ ]:
#final.dropna(inplace=True)
final.isna().any().any()

False

In [ ]:
# add returns column
final.loc[:,'spf_ret'] = final.loc[:,'spf_adj close'].pct_change().fillna(method='bfill')
(1+final['spf_ret']).prod()-1

1.6883144450022947

In [ ]:
# basic descriptives of SP500 futures returns
final.loc[:,'spf_ret'].describe()

count    5472.000000
mean        0.000258
std         0.012444
min        -0.103765
25%        -0.004477
50%         0.000720
75%         0.005731
max         0.141134
Name: spf_ret, dtype: float64

In [ ]:
#create target variable 
target = final.loc[:,'spf_ret'].quantile(q=.05)
target

-0.01925306382745108

In [ ]:
final.loc[:,'target'] = final.loc[:,'spf_ret'].apply(lambda x: 0 if x > target else 1)
final.to_csv(f'final_{day.day}_{day.month}_{day.year}.csv')
# check the size of the target length
assert round(final.loc[:,'target'].sum()/final.shape[0],2)==.05

# Save DF

In [ ]:
day = datetime.today()
final.to_csv(f'final_{day.day}_{day.month}_{day.year}.csv')